In [ ]:
# NOTE: sep 2022
# This script does no longer work correctly on the wikipedia page of the exercise, since it has changed over time
# since this script has been developed in 2019
# It might nevertheless be useful as a starting point

In [ ]:
import re
import urllib
import time

import requests
from bs4 import BeautifulSoup
headers = {'user-agent': 'scrapingCourseBot'}

This convenience function finds the text directly inside a BeautifulSoup tag, ignoring any text in descendant tags. It also cleans up the text by stripping leading and trailing whitespace. Such whitespace  is irrelevant in HTML.

In [ ]:
def find_text_only(tag):
    text = tag.find(text=True, recursive=False)
    return text.strip()

First part of the exercise: find country and song name for all songs in the final of the 2018 Eurovision song contest. This data can be found in [https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2018](https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2018). 

In [ ]:
url = "https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2018"
page = requests.get(url, headers=headers)
print(url, page.status_code)

This page contains a table with all the data in the section "Participating Countries", in the "Final" subsection. This subsection can easily be found by its id, which is also "Final"; the table is contained in the first &lt;table&gt; HTML element after that.

In [ ]:
soup = BeautifulSoup(page.text, "lxml")

final = soup.find(id = "Final")
table = final.findNext("table")

Each column in this table contains a separate variable, while each row in the table represents one song. The first row contains column headers, with citations for each header. We process this table one row at a time, starting with the row with column headers. We extract all column headers, and make sure to get rid of the unwanted citations.

In [ ]:
# Each table row is contained in a <tr> tag.
rows = table.find_all("tr")

# The column headers are inside the first table row, inside <th> tags.
firstrow_cells = rows[0].find_all("th")
headers = [ find_text_only(cell) for cell in firstrow_cells ]
print(headers)

Now we extract the variables we want from the remaining rows in the table. For each row, we loop simultaneously through the column headers and the cells in the row, so that we know which variable is contained within each cell. Each cell is contained in a &lt;td&gt; tag.

[as a sidenote: I only got the idea of looping through column headers and table cells simultaneously, using zip(), several days after the course had ended. My original solution was quite a bit more complex and therefore much more time-consuming to get right]

The question only asked for countries and song names, but we also extract links to individual song pages and the number of points per song, in preparation for the second part of the exercise.

In [ ]:
results = []
for row in rows[1:]:
    row_result = {}
    row_cells = row.find_all("td")
    for header, cell_content in zip(headers, row_cells):
        if header == "Country":
            # We only store the text within the a tag inside the cell,
            # because the cell also contains the country flag, which we do not want.
            row_result["country"] = cell_content.a.text
        elif header == "Song":
            # Again, we only store the text within the a tag inside the cell,
            # this time because the song titles are between quotes, and we do not want those.
            row_result["songname"] = cell_content.a.text
            # We also store the url to the song page, for future reference.
            # We use urllib.parse.urljoin to convert these into absolute urls.
            # This is more robust than simply adding "www.wikipedia.org" in front.
            row_result["link"] = urllib.parse.urljoin(url, cell_content.a["href"])
        elif header == "Points":
            # We only store the text within the cell itself, not its children.
            # The reason here is that these cells also contain a hidden tag 
            # with a sort key, and we do not want those.
            # We also convert the result to integer, as we need it as such later.
            row_result["points"] = int(find_text_only(cell_content))
    results.append(row_result)
    print(row_result)

Second part of the exercise: gather release dates, song lengths and songwriters from the
individual song pages, for those songs that got at least 100 points.

The individual song pages all contain a wikipedia infobox in the sidebar on the right with the required data. The HTML structure of this infobox is the same for all song pages, which makes scraping the required data much simpler. The table itself has the class "vevent". Each row is contained in a &lt;tr&gt; tag, and each row contains 1 variable. Within a row, the variable name is contained in a &lt;th&gt; tag, while its value is contained in a &lt;td&gt; tag.

In [ ]:
# Second part of the exercise: gather release dates, song lengths and songwriters from the
# individual song pages, but only for those songs that got at least 100 points
for row_result in results:
    if row_result["points"] < 100: 
        continue

    # We visist the song's page. We sleep before the request instead of after the request:
    # On the one hand, this inserts a pause after the previous page read, and on the other
    # hand, this avoids having to wait an extra second after the final request.
    time.sleep(1)
    page = requests.get(row_result["link"])
    print(row_result["link"], page.status_code) # As before, we print a log message
    soup = BeautifulSoup(page.text, "lxml")

    # The data we want is in the table in the sidebar on the right. This table has
    # class name "vevent".
    detail_table = soup.find("table", {"class": "vevent"})

    # This table has a separate variable in each row, with one <th> tag containing
    # the variable name, and one <td> tag containing its value.
    # We therefore simply loop over all rows
    rows = detail_table.find_all("tr")
    for row in rows:
        label_cell = row.th
        value_cell = row.td
        if label_cell is not None and value_cell is not None:
            label = label_cell.text
            if label == "Released":
                # We use find_text_only() here, because it is more robust
                # than simply copying all text. Also, for some songs, the release
                # date is accompanied by a wikipedia citation, which we do not want.
                row_result["released"] = find_text_only(value_cell)
            elif label == "Length":
                # Most song lengths are strings with format "min:sec". One might
                # be tempted to parse this string here to store the song length
                # as a number of seconds, but we choose not to do so: if
                # necessary, we can still do this later.
                row_result["length"] = value_cell.text
            elif label == "Songwriter(s)":
                # There may be more than 1 songwriter. For most songs,
                # each songwriter is inside a separate <li> tag within
                # an <ul> tag. But some songs have a single songwriter 
                # directly in the cell, without containing <ul> tag
                writer_list = value_cell.ul
                if writer_list is None:
                    # Usually there's a single name here, but there's one
                    # case with multiple names separated by commas.
                    row_result["songwriters"] = [ name.strip() for name in value_cell.text.split(",") ]
                else:
                    # We use all text inside the <li> tags. We believe 
                    # that in this case, this is more appropriate.
                    # We do not want to ignore the text in descendant tags
                    # because if we would do so, we would lose names 
                    # inside hyperlinks.
                    row_result["songwriters"] = [ name.text for name in writer_list.find_all("li") ]

In [ ]:
for result in results:
    print(result)